# Using Keras RL

In [1]:
import gym_fishing
import gym
import math
import random
from math import floor
import bokeh.plotting
import numpy as np
import bokeh.io
from itertools import count
import bokeh.plotting
from bokeh.models import Span
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from rl.memory import SequentialMemory
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy

bokeh.io.output_notebook()

Using TensorFlow backend.


Loading BokehJS ...

For proof of concept, I am going to use the DQN RL agent for the CartPole environment.

In [2]:
# Instatiating the environment then making a step to show the output
cartpole_env = gym.make('CartPole-v0')
cartpole_env.reset()
for i in range(10):
    print(cartpole_env.step(0))

(array([-0.03449747, -0.24023462, -0.0044588 ,  0.26833139]), 1.0, False, {})
(array([-0.03930216, -0.43529265,  0.00090782,  0.55960465]), 1.0, False, {})
(array([-0.04800801, -0.63042734,  0.01209992,  0.85257345]), 1.0, False, {})
(array([-0.06061656, -0.82571213,  0.02915139,  1.14903649]), 1.0, False, {})
(array([-0.0771308 , -1.02120222,  0.05213212,  1.45071633]), 1.0, False, {})
(array([-0.09755485, -1.21692459,  0.08114644,  1.75922093]), 1.0, False, {})
(array([-0.12189334, -1.41286626,  0.11633086,  2.07599717]), 1.0, False, {})
(array([-0.15015066, -1.60896022,  0.1578508 ,  2.40227427]), 1.0, False, {})
(array([-0.18232987, -1.80506894,  0.20589629,  2.73899579]), 1.0, False, {})
(array([-0.21843125, -2.00096552,  0.26067621,  3.08674082]), 1.0, True, {})


/Users/MLapeyrolerie/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Now I make a FF neural net with 2 hidden layers each wih width 128 neurons.

In [3]:
def agent(shape, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, shape)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

Now, training the agent.

In [4]:
model = agent(4, 2)
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannGumbelQPolicy()

dqn = DQNAgent(model=model, policy=policy, nb_actions=2, memory=memory)
dqn.compile('adam', metrics = ['mae'])
dqn.fit(cartpole_env, nb_steps = 1000, visualize = False, verbose = 1)

Training for 1000 steps ...
Interval 1 (0 steps performed)
  994/10000 [=>............................] - ETA: 5s - reward: 1.0000done, took 0.656 seconds


Testing the agent

In [5]:
_ = dqn.test(cartpole_env, nb_episodes = 5, visualize= False)

Testing for 5 episodes ...
Episode 1: reward: 43.000, steps: 43
Episode 2: reward: 119.000, steps: 119
Episode 3: reward: 40.000, steps: 40
Episode 4: reward: 61.000, steps: 61
Episode 5: reward: 57.000, steps: 57


This all looks good.

Now, doing the same for the fishing gym environment, I first instatiate the gym.

In [6]:
# Instatiating the environment then making a step to show the output
env = gym.make('fishing-v0')
env.reset()
env.step(0)

(array([99686]), 0.0, False, {})

Here, I train the DQN agent. Only changing the `input_shape` from above.

In [7]:
model = agent(1, len(env.action_space))
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannGumbelQPolicy()

dqn = DQNAgent(model=model, policy=policy, nb_actions=len(env.action_space), memory=memory)
dqn.compile('adam', metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_2_input to have shape (1, 1) but got array with shape (1, 4)